In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
from accelerate import Accelerator
import torch.optim as optim

In [5]:
import time

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [9]:
##### 
'''
Model 1 is be multivariate timeseries, each variate represents unique country,store, product

We have 3 countries, 2 stores and 3 products, this means we need to predict 3*2*3 18 values representing each.
'''


'\nModel 1 is be multivariate timeseries, each variate represents unique country,store, product\n\nWe have 3 countries, 2 stores and 3 products, this means we need to predict 3*2*3 18 values representing each.\n'

In [10]:
torch.cuda.is_available()

True

In [11]:
original_train_df = pd.read_csv('./data/train.csv', parse_dates=['date'])
original_test_df = pd.read_csv('./data//test.csv', parse_dates=['date'])
gdp_df = pd.read_csv('./data/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv',
                    index_col='year')

original_train_df.head(2)

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520


In [12]:
original_train_df['product'].unique()

array(['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker'], dtype=object)

In [13]:
def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

In [14]:
gdp_df

,GDP_Finland,GDP_Norway,GDP_Sweden
year,,,
2015,234.440,385.802,505.104
2016,240.608,368.827,515.655
2017,255.017,398.394,541.019
2018,275.580,437.000,555.455
2019,268.782,405.510,533.880


# Feature engineering

In [15]:
# Feature engineering
def get_gdp(row):
    """Return the GDP based on row.country and row.date.year"""
    country = 'GDP_' + row.country
    return gdp_df.loc[row.date.year, country]

le_dict = {feature: LabelEncoder().fit(original_train_df[feature]) for feature in ['country', 'product', 'store']}

def engineer(df):
    """Return a new dataframe with the engineered features"""
    
    new_df = pd.DataFrame({'gdp': df.apply(get_gdp, axis=1),
                           'dayofyear': df.date.dt.dayofyear,
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    new_df.loc[(df.date.dt.year != 2016) & (df.date.dt.month >=3), 'dayofyear'] += 1 # fix for leap years
    
    for feature in ['country', 'product', 'store']:
        new_df[feature] = le_dict[feature].transform(df[feature])
        
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df['days_from_easter'] = (df.date - easter_date).dt.days.clip(-5, 65)
    
    # Last Sunday of May (Mother's Day)
    sun_may_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-5-31')),
                                         2016: pd.Timestamp(('2016-5-29')),
                                         2017: pd.Timestamp(('2017-5-28')),
                                         2018: pd.Timestamp(('2018-5-27')),
                                         2019: pd.Timestamp(('2019-5-26'))})
    #new_df['days_from_sun_may'] = (df.date - sun_may_date).dt.days.clip(-1, 9)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df['days_from_wed_jun'] = (df.date - wed_june_date).dt.days.clip(-5, 5)
    
    # First Sunday of November (second Sunday is Father's Day)
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df['days_from_sun_nov'] = (df.date - sun_nov_date).dt.days.clip(-1, 9)
    
    return new_df

train_df = engineer(original_train_df)
train_df['date'] = original_train_df.date # used in GroupKFold
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
train_df['target'] = np.log(train_df['num_sold'] / train_df['gdp'])
test_df = engineer(original_test_df)
test_df['date'] = original_test_df.date # used in GroupKFold

In [16]:
test_df

,gdp,dayofyear,wd4,wd56,country,product,store,days_from_easter,days_from_wed_jun,days_from_sun_nov,date
0,268.782,1,False,False,0,1,0,-5,-5,-1,2019-01-01
1,268.782,1,False,False,0,0,0,-5,-5,-1,2019-01-01
2,268.782,1,False,False,0,2,0,-5,-5,-1,2019-01-01
3,268.782,1,False,False,0,1,1,-5,-5,-1,2019-01-01
4,268.782,1,False,False,0,0,1,-5,-5,-1,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...
6565,533.880,366,False,False,2,0,0,65,5,9,2019-12-31
6566,533.880,366,False,False,2,2,0,65,5,9,2019-12-31
6567,533.880,366,False,False,2,1,1,65,5,9,2019-12-31
6568,533.880,366,False,False,2,0,1,65,5,9,2019-12-31


In [17]:
in_features = ['dayofyear', 'days_from_easter', 'days_from_sun_nov', 'days_from_wed_jun', 'wd4', 'wd56','country','store','product', 'target']

In [18]:
train_df

,gdp,dayofyear,wd4,wd56,country,product,store,days_from_easter,days_from_wed_jun,days_from_sun_nov,date,num_sold,target
0,234.440,1,False,False,0,1,0,-5,-5,-1,2015-01-01,329.0,0.338858
1,234.440,1,False,False,0,0,0,-5,-5,-1,2015-01-01,520.0,0.796629
2,234.440,1,False,False,0,2,0,-5,-5,-1,2015-01-01,146.0,-0.473593
3,234.440,1,False,False,0,1,1,-5,-5,-1,2015-01-01,572.0,0.891939
4,234.440,1,False,False,0,0,1,-5,-5,-1,2015-01-01,911.0,1.357343
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,555.455,366,False,False,2,0,0,65,5,9,2018-12-31,823.0,0.393169
26294,555.455,366,False,False,2,2,0,65,5,9,2018-12-31,250.0,-0.798327
26295,555.455,366,False,False,2,1,1,65,5,9,2018-12-31,1004.0,0.591960
26296,555.455,366,False,False,2,0,1,65,5,9,2018-12-31,1441.0,0.953305


In [19]:
grp_df_train = train_df.groupby('date', as_index=False).agg(target = ('target', list),
                             dayofyear =('dayofyear', list),
                             wd4 =('wd4', list),
                             wd56 = ('wd56', list),
                             country =('country', list),
                             product =('product', list),
                             store =('store', list),
                             days_from_easter =('days_from_easter', list),
                             days_from_wed_jun =('days_from_wed_jun', list),
                             days_from_sun_nov =('days_from_sun_nov', list),
                            )

train_df2 = pd.DataFrame({'date': grp_df_train['date'].values,
              'features':grp_df_train.apply(lambda x: np.array([np.array(x[f]) for f in in_features]), axis=1)  
})

In [20]:
grp_df_test = test_df.groupby('date', as_index=False).agg(
                             dayofyear =('dayofyear', list),
                             wd4 =('wd4', list),
                             wd56 = ('wd56', list),
                             country =('country', list),
                             product =('product', list),
                             store =('store', list),
                             days_from_easter =('days_from_easter', list),
                             days_from_wed_jun =('days_from_wed_jun', list),
                             days_from_sun_nov =('days_from_sun_nov', list),
                            )

test_df2 = pd.DataFrame({'date': grp_df_test['date'].values,
              'features':grp_df_test.apply(lambda x: np.array([np.array(x[f]) for f in in_features if f!='target']), axis=1)  
})

#### Config #####

In [21]:
config = {
    'seq_length' : 60,
    'num_epochs' : 200,
    'lr' : 0.001,
    'input_size' : 180,
    'hidden_size' : 360,
    'num_layers' : 2,
    'num_classes' :18, ## This is  output dimension
    'train_shuffle': True,
    'val_shuffle': True,
    'batch_size' : 30,
    'best_model_name' : 'lstm_tsp_mlp_head_drpOut_1.bin',
    'bidirectional' : False,
    'only_last_hidden': False
}
# config_lr = {'T_max':20,
#              'eta_min':0
#             }

# Training #

### Dataloader ###

#### Make sequences ####

In [22]:
def sliding_windows(data, seq_length):
        x = []
        y = []

        for i in range(len(data)-seq_length-1):
            _x = data[i:(i+seq_length),:].transpose(0,2,1).reshape(seq_length,-1)
            _y = data[i+seq_length,-1]
            x.append(_x)
            y.append(_y)

        return np.array(x),np.array(y)
    
def make_sequences(df,seq_length):
    data = np.rollaxis(np.dstack(df['features'].values.tolist()),-1)
    print('Data Shape', data.shape)
    
    x, y = sliding_windows(data, seq_length)

    print('X,y shapes', x.shape,y.shape)
    
    return x,y
    

In [23]:
X,y = make_sequences(train_df2,config['seq_length'])

Data Shape (1461, 10, 18)
X,y shapes (1400, 60, 180) (1400, 18)


In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
class TPSDataset(Dataset):
    
    def __init__(self, x,y):
        """
        Args:
        """
        self.x=x
        self.y=y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        sample = [torch.Tensor(self.x[idx]), torch.Tensor(self.y[idx])]
        return sample

## Model ##

In [26]:
num_epochs = config['num_epochs']
lr = config['lr']
input_size = config['input_size']
hidden_size = config['hidden_size']
num_layers = config['num_layers']
num_classes = config['num_classes']
seq_length = config['seq_length']
bidirectional = config['bidirectional']
only_last_hidden = config['only_last_hidden']

In [27]:
class LSTMTpsModel(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers,seq_length):
        super(LSTMTpsModel, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        
        if bidirectional:
            m=2
        else:
            m=1
        
        if only_last_hidden:
            input_dim = hidden_size*m
        else:
            input_dim = self.seq_length*hidden_size*m
        
        self.fc = nn.Sequential(nn.Linear(input_dim, input_dim//8),
                                # nn.BatchNorm1d(num_features=input_dim//8),
                                nn.Dropout(0.2),
                                nn.ReLU(),
                                
                                nn.Linear(input_dim//8, input_dim//16),
                                # nn.BatchNorm1d(num_features=input_dim//16),
                                nn.Dropout(0.2),
                                nn.ReLU(),
                                
                                nn.Linear(input_dim//16, input_dim//32),
                                # nn.BatchNorm1d(num_features=input_dim//32),
                                nn.Dropout(0.2),
                                nn.ReLU(),
                                nn.Linear(input_dim//32, self.num_classes)
                                )

    def forward(self, x):
        # Propagate input through LSTM
        h_out, (_, _) = self.lstm(x)
        if only_last_hidden:
            h_out = h_out[:,-1:,:]
        h_out = h_out.flatten(start_dim=1)
        
        out = self.fc(h_out)
        
        return out

In [28]:
def run(model,train_dl,val_dl):
    def evaluate(model,valid_loader):
        model.eval()
        valid_loss = 0
        rec_loss = 0
        with torch.no_grad():
            for i, inputs in enumerate(valid_loader):
                dataX = inputs[0]
                dataY = inputs[1]
                outputs = model(dataX)
                loss = criterion(outputs, dataY)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)
        return valid_loss
    
    def train_and_evaluate_loop(train_loader,model,optimizer,criterion,epoch,lr_scheduler=None,valid_loader=None, best_loss=99999):
        train_loss = 0
        for i, inputs in enumerate(train_loader):
            optimizer.zero_grad()
            model.train()
            
            dataX = inputs[0]
            dataY = inputs[1]
            outputs = model(dataX)
            loss = criterion(outputs, dataY)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if lr_scheduler:
                lr_scheduler.step()
        
        train_loss /= len(train_loader)
        if valid_loader:
            valid_loss = evaluate(model,valid_loader) 
            print(f"Epoch:{epoch} |Train Loss:{train_loss}|Valid Loss:{valid_loss}")
            if valid_loss <= best_loss:
                print(f"{g_}Loss Decreased from {best_loss} to {valid_loss}{sr_}")

                best_loss = valid_loss
                torch.save(model.state_dict(), config['best_model_name'])
        else:
            print(f"Epoch:{epoch} |Train Loss:{train_loss}")
            
                    
        return best_loss
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")

    
    
    optimizer = optim.Adam(model.parameters(),lr=config['lr'],amsgrad=False)
    criterion = torch.nn.MSELoss()
    lr_scheduler = None
    # lr_scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **config_lr)

    model,train_dl,val_dl,optimizer,lr_scheduler,criterion = accelerator.prepare(model,train_dl,val_dl,optimizer,lr_scheduler,criterion)

    best_loss = 9999999
    start_time = time.time()
    for epoch in range(config["num_epochs"]):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,model,optimizer,criterion,epoch,lr_scheduler,valid_loader=val_dl, best_loss=best_loss)
        
        end_time = time.time()
        print(f"{m_}Time taken by epoch {epoch} is {end_time-start_time:.2f}s{sr_}")
        start_time = end_time
        
    return best_loss, model

In [29]:
model = LSTMTpsModel(num_classes, input_size, hidden_size, num_layers,seq_length)

In [30]:
train_dl = DataLoader(TPSDataset(X_train, y_train), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=2)
val_dl = DataLoader(TPSDataset(X_val, y_val), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=2)

In [31]:
best_loss, model = run(model,train_dl,val_dl)

cuda is used
Epoch Started:0
Epoch:0 |Train Loss:0.18878438235505632|Valid Loss:0.037116770073771475
Loss Decreased from 9999999 to 0.037116770073771475
Time taken by epoch 0 is 2.96s
Epoch Started:1
Epoch:1 |Train Loss:0.04778810389535992|Valid Loss:0.030099331773817538
Loss Decreased from 0.037116770073771475 to 0.030099331773817538
Time taken by epoch 1 is 1.81s
Epoch Started:2
Epoch:2 |Train Loss:0.039293781609127394|Valid Loss:0.029937287978827953
Loss Decreased from 0.030099331773817538 to 0.029937287978827953
Time taken by epoch 2 is 1.78s
Epoch Started:3
Epoch:3 |Train Loss:0.03586163259062328|Valid Loss:0.026530300080776215
Loss Decreased from 0.029937287978827953 to 0.026530300080776215
Time taken by epoch 3 is 1.93s
Epoch Started:4
Epoch:4 |Train Loss:0.03951118768830048|Valid Loss:0.038379752077162266
Time taken by epoch 4 is 1.17s
Epoch Started:5
Epoch:5 |Train Loss:0.035010789472021554|Valid Loss:0.026493257191032172
Loss Decreased from 0.026530300080776215 to 0.026493257